__Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto__

*Part 1: Use Python libraries to scrape Wikipedia Website for Postal Code, Borough, and Neighborhood*
<br>
Source: 
<a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">Wikipedia Page</a>

In [1]:
import numpy as np
import pandas as pd
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [2]:
table = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
table_df = table[0]
table_df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [3]:
table_df.rename(columns = {'Postal code': 'PostalCode'}, inplace = True)

*Ignore rows with Borough = 'Not assigned'*

In [4]:
table_df = table_df.loc[table_df['Borough'] != 'Not assigned']

*Combine Rows that have more than one neighborhood corresponding to a single borough, into a signle row. Each neighborhood muct be separted by commas. Note that current version of wikipedia page has already aggregated the neighborhoods. Here I am simply replacing "/" separator by "," separator. If this had not been done, a combination of set and join functions could acheive desired results. For more information, visit: <a href="https://stackoverflow.com/questions/54216702/pandas-grouping-by-column-one-and-adding-comma-separated-entries-from-column-two">Stack Overflow Post</a>*

In [5]:
table_df["Neighborhood"] = table_df["Neighborhood"].str.replace("/",",")

C:\Users\Rubenka\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [6]:
table_df.shape

(103, 3)

In [7]:
#conda install -c conda-forge geocoder
# This needs to be installed only once if using geocoder to access latitude and longitude information of Toronto neighborhoods

In [8]:
# Used csv file instead instead of geocoder application

*Add latitude and longitude information to dataframe*

In [9]:
lat_long_data = pd.read_csv('Geospatial_Coordinates.csv')
lat_long_data.set_index('PostalCode',inplace = True)
table_df.set_index('PostalCode',inplace = True)

In [10]:
# used inner join to combine the Wikipedia table with latitude longitude information table
result = pd.concat([table_df, lat_long_data], axis = 1, join = 'inner')

In [11]:
result.head()

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
